# Лабораторная работа №2. Парсинг Yahoo Finance с использованием Playwright

**Цель работы:** научиться извлекать данные с веб-страниц, которые подгружают контент динамически с помощью JavaScript. Освоить использование современной библиотеки `Playwright` для управления браузером и языка `XPath` для точного поиска элементов на странице.

**Объект парсинга:** страница "Most Active" сайта [finance.yahoo.com](https://finance.yahoo.com/most-active).

## Часть 1: Теоретические основы XPath

### Что такое XPath?

**XPath (XML Path Language)** — это язык запросов, предназначенный для навигации по элементам и атрибутам в XML и HTML документах. Представьте HTML-страницу как дерево, состоящее из вложенных друг в друга тегов. XPath предоставляет синтаксис, похожий на путь в файловой системе, чтобы точно указать, какой элемент или группу элементов мы хотим найти в этом дереве.

Для веб-скрапинга XPath является мощным инструментом, позволяющим создавать надежные "селекторы" (указатели) для извлечения нужных данных, даже если структура страницы сложная.

### Основной синтаксис XPath

| Выражение | Описание | Пример |
|:---|:---|:---|
| `//` | Выбирает узлы в документе, начиная с текущего, которые соответствуют выбору, где бы они ни находились. | `//table` (найти все таблицы на странице) |
| `tagname` | Выбирает все узлы с заданным именем тега. | `//tr` (найти все строки таблицы) |
| `.` | Выбирает текущий узел. | - |
| `@` | Выбирает атрибуты. | `//button[@name='agree']` (найти кнопку с атрибутом `name`, равным 'agree') |
| `[...]` | Предикат для фильтрации. Позволяет указать точные условия. | `//tr[@data-testid='data-table-v2-row']` (найти строку с атрибутом `data-testid`) |
| `text()` | Выбирает текстовое содержимое узла. | - |
| `contains()` | Функция, которая позволяет искать частичное совпадение текста в атрибуте или содержимом. | `//div[contains(@class, 'table-container')]` (найти div, у которого класс *содержит* 'table-container') |

В нашей практической части мы будем использовать комбинации этих выражений для поиска таблицы и извлечения данных из ячеек.

## Часть 2. Практика - Парсинг Yahoo Finance с помощью Playwright

В этой части мы напишем скрипт, который будет:
1. Управлять браузером Chromium в среде Google Colab с помощью Playwright.
2. Открывать страницу `most-active` на Yahoo Finance.
3. Дожидаться полной загрузки динамической таблицы.
4. Собирать данные об акциях.
5. Сохранять результат в Excel-файл.

In [ ]:
# Шаг 2.1. Установка Playwright и его зависимостей

# Устанавливаем саму библиотеку
!pip install playwright

# Скачиваем браузеры, которыми Playwright будет управлять (в данном случае Chromium)
!playwright install

In [ ]:
# Шаг 2.2: Запуск скрипта для сбора данных с Yahoo Finance

import pandas as pd
from playwright.async_api import async_playwright
import asyncio

# Создаем глобальный DataFrame, чтобы он был доступен после выполнения функции
df = pd.DataFrame()

async def main():
    global df
    async with async_playwright() as p:
        print("Запуск браузера...")
        # В Colab браузер запускается в режиме headless (без графического интерфейса)
        browser = await p.chromium.launch()
        page = await browser.new_page()

        url = 'https://finance.yahoo.com/most-active'
        print(f"Переход на страницу: {url}")
        await page.goto(url, timeout=90000)

        # Блок обработки cookie на всякий случай
        try:
            print("Поиск окна согласия на cookie...")
            agree_button_selector = '//button[@name="agree"]'
            await page.locator(agree_button_selector).click(timeout=10000)
            print("Кнопка согласия нажата.")
        except Exception:
            print("Окно согласия не найдено или уже принято. Продолжаем...")

        # Ожидание появления первой строки таблицы - это подтверждение, что данные загрузились
        print("Ожидание загрузки данных в таблицу...")
        await page.wait_for_selector('//tr[@data-testid="data-table-v2-row"]', timeout=60000)
        print("Данные в таблице загружены.")

        # Извлечение данных с помощью правильных селекторов
        print("Извлечение данных...")
        symbols = await page.locator('//td[@data-testid-cell="ticker"]').all_text_contents()
        names = await page.locator('//td[@data-testid-cell="companyshortname.raw"]').all_text_contents()
        prices = await page.locator('//td[@data-testid-cell="intradayprice"]').all_text_contents()
        changes = await page.locator('//td[@data-testid-cell="intradaypricechange"]').all_text_contents()
        percent_changes = await page.locator('//td[@data-testid-cell="percentchange"]').all_text_contents()
        volumes = await page.locator('//td[@data-testid-cell="dayvolume"]').all_text_contents()

        print("Закрытие браузера...")
        await browser.close()

        # Создаем DataFrame
        min_len = min(len(symbols), len(names), len(prices), len(changes), len(percent_changes), len(volumes))
        data = {
            'Symbol': symbols[:min_len], 'Name': names[:min_len], 'Price': prices[:min_len],
            'Change': changes[:min_len], 'Percent Change': percent_changes[:min_len], 'Volume': volumes[:min_len]
        }
        df = pd.DataFrame(data)

        # Сохранение в файл
        output_filename = 'yahoo_most_active_playwright.xlsx'
        df.to_excel(output_filename, index=False)
        print(f"\nДанные успешно сохранены в файл: {output_filename}")
        print("Файл можно найти в панели слева ('Файлы').")

# Запускаем асинхронную функцию (в Colab можно использовать 'await' на верхнем уровне)
await main()


## Часть 3. Анализ полученных данных

Теперь, когда у нас есть файл `yahoo_most_active_playwright.xlsx`, мы можем загрузить его и провести базовый анализ.

**Внимание:** убедитесь, что файл `yahoo_most_active_playwright.xlsx` появился в файловом менеджере Colab слева, прежде чем выполнять следующую ячейку.

In [ ]:
# Шаг 3.1: Загрузка и первичный осмотр данных

import pandas as pd

try:
    file_path = 'yahoo_most_active_playwright.xlsx'
    df = pd.read_excel(file_path)

    print(f"Файл '{file_path}' успешно загружен.")
    print(f"Размер таблицы: {df.shape[0]} строк, {df.shape[1]} колонок.")

    print("\nПервые 5 строк таблицы:")
    display(df.head())

    print("\nТипы данных в колонках:")
    df.info()

except FileNotFoundError:
    print(f"Ошибка: файл '{file_path}' не найден. Убедитесь, что предыдущая ячейка отработала успешно и файл создан.")

In [ ]:
# Шаг 3.2: Предварительная обработка данных

df_clean = df.copy()

# 1. Очистка 'Price': в этой колонке может быть лишняя информация (например, '30.57 +5.67 (+22.77%)').
# Нам нужно только первое число.
df_clean['Price'] = df_clean['Price'].astype(str).str.split(' ').str[0]

# 2. Очистка 'Percent Change': убираем скобки, знаки '+' и '%'
df_clean['Percent Change'] = df_clean['Percent Change'].astype(str).str.replace(r'[()%+]', '', regex=True)

# 3. Очистка 'Volume': конвертируем 'M' (миллионы) и 'B' (миллиарды) в числа
def convert_volume(volume_str):
    volume_str = str(volume_str).strip()
    if volume_str.endswith('M'):
        return float(volume_str[:-1]) * 1_000_000
    elif volume_str.endswith('B'):
        return float(volume_str[:-1]) * 1_000_000_000
    elif volume_str.endswith('T'):
        return float(volume_str[:-1]) * 1_000_000_000_000
    return pd.to_numeric(volume_str, errors='coerce')

df_clean['Volume'] = df_clean['Volume'].apply(convert_volume)

# 4. Преобразование колонок в числовой формат
cols_to_numeric = ['Price', 'Change', 'Percent Change', 'Volume']
for col in cols_to_numeric:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# Удаляем строки, где могли возникнуть ошибки преобразования (стали NaN)
df_clean.dropna(subset=cols_to_numeric, inplace=True)

# 5. Преобразуем Volume в целое число для наглядности
df_clean['Volume'] = df_clean['Volume'].astype('int64')

print("--- ДАННЫЕ ПОСЛЕ ОЧИСТКИ ---")
df_clean.info()
print(df_clean.head())

In [ ]:
# Шаг 3.3: Анализ и визуализация данных

import matplotlib.pyplot as plt
import seaborn as sns

# Настройки для графиков
sns.set_style("whitegrid")

# --- Анализ 1: Топ-10 акций по объему торгов ---
top_10_volume = df_clean.sort_values(by='Volume', ascending=False).head(10)

plt.figure(figsize=(10, 6))
ax1 = sns.barplot(x='Volume', y='Symbol', data=top_10_volume, palette='viridis')
ax1.set_title('Топ-10 акций по объему торгов', fontsize=16)
ax1.set_xlabel('Объем', fontsize=12)
ax1.ticklabel_format(style='plain', axis='x') # Отключаем научную нотацию для оси X
ax1.set_ylabel('Тикер', fontsize=12)
plt.show()

# --- Анализ 2: Лидеры роста и падения ---
top_5_gainers = df_clean.sort_values(by='Percent Change', ascending=False).head(5)
top_5_losers = df_clean.sort_values(by='Percent Change', ascending=True).head(5)
gainers_losers = pd.concat([top_5_gainers, top_5_losers])

plt.figure(figsize=(10, 6))
# Создаем палитру: зеленый для положительных, красный для отрицательных
palette = ['#32a852' if x > 0 else '#d42c2c' for x in gainers_losers['Percent Change']]
ax2 = sns.barplot(x='Percent Change', y='Symbol', data=gainers_losers, palette=palette)
ax2.set_title('Топ-5 лидеров роста и падения (%)', fontsize=16)
ax2.set_xlabel('Изменение (%)', fontsize=12)
ax2.set_ylabel('Тикер', fontsize=12)
plt.show()

print("\n--- Лидеры роста ---")
display(top_5_gainers[['Symbol', 'Name', 'Price', 'Percent Change']])

print("\n--- Лидеры падения ---")
display(top_5_losers[['Symbol', 'Name', 'Price', 'Percent Change']])

## Выводы

В ходе выполнения лабораторной работы мы успешно решили задачу парсинга данных с динамического веб-сайта Yahoo Finance с использованием современной библиотеки Playwright.

1.  **Применили знания XPath** для создания надежных селекторов, нацеленных на атрибуты `data-testid`, что является современной практикой для тестируемых веб-приложений.
2.  **Настроили и использовали Playwright** в Google Colab для асинхронной автоматизации работы с браузером. Этот подход показал себя как более стабильный и быстрый по сравнению с традиционным Selenium для подобных задач.
3.  **Извлекли данные** по самым активным акциям и сохранили их в структурированном виде (Excel-файл).
4.  **Провели комплексную предварительную обработку данных** с помощью Pandas, преобразовав текстовые значения объемов (например, '500M') и процентов в числовые форматы, пригодные для анализа.
5.  **Выполнили базовый анализ и визуализацию**, определив лидеров по объему торгов, а также лидеров роста и падения в процентном соотношении.